# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key 

geoapify_key = "ee0e93b9d1014aea9a86b537e7201403"

In [19]:
pip install geoviews hvplot pandas


Note: you may need to restart the kernel to use updated packages.


In [20]:

city_data_df = pd.read_csv("output_data/cities.csv")

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,81.23,60,7,12.35,YE,1707896652
1,1,isafjordur,66.0755,-23.1240,33.84,69,75,12.66,IS,1707896653
2,2,nadym,65.5333,72.5167,-20.99,92,7,15.30,RU,1707896654
3,3,ushuaia,-54.8000,-68.3000,47.86,61,75,4.61,AR,1707896469
4,4,port-aux-francais,-49.3500,70.2167,47.05,91,100,23.35,TF,1707896655


In [21]:
import geoviews as gv

anoje_coordinates_module6= {
    "Lat": city_data_df["Lat"],
    "Lng": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Humidity": city_data_df["Humidity"]
}

co = pd.DataFrame(anoje_coordinates_module6)

final = gv.Dataset(co, kdims=["Lng", "Lat"], vdims=["City", "Humidity"])

tiles = gv.tile_sources.OSM()

points = final.to(gv.Points, ["Lng", "Lat"], ["Humidity", "City"])

output = tiles * points.opts(
    width=1500,
    height=1100,
    size="Humidity",
    color="City",
    cmap="viridis"
)

output



c:\Users\anoje\anaconda3\Lib\site-packages\holoviews\core\util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.5, use 'transform_reference' instead.
  value = param_value_if_widget(value)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
print(city_data_df.columns)

hotel_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 5) 
].dropna()

display(hotel_df)



Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
280,280,grand falls-windsor,48.9332,-55.6649,25.50,89,100,3.00,CA,1707896552
383,383,burang,30.2500,81.1667,26.38,43,15,4.59,CN,1707896891


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
display(hotel_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
280,280,grand falls-windsor,48.9332,-55.6649,25.50,89,100,3.00,CA,1707896552
383,383,burang,30.2500,81.1667,26.38,43,15,4.59,CN,1707896891


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "category": "lodging",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Hotel Currently Searching ....")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"point:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found !!!!!!! $"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
display(hotel_df)


Hotel Currently Searching ....
grand falls-windsor - nearest hotel: No hotel found !!!!!!! $
burang - nearest hotel: No hotel found !!!!!!! $


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
280,280,grand falls-windsor,48.9332,-55.6649,25.50,89,100,3.00,CA,1707896552,No hotel found !!!!!!! $
383,383,burang,30.2500,81.1667,26.38,43,15,4.59,CN,1707896891,No hotel found !!!!!!! $


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
coordinates = {
    "Lat": hotel_df["Lat"],
    "Lng": hotel_df["Lng"],
    "City": hotel_df["City"],
    "Humidity": hotel_df["Humidity"],
    "Hotel Name": hotel_df["Hotel Name"],
    "Country": hotel_df["Country"]
}

co = pd.DataFrame(coordinates)

gdf = gv.Dataset(co, kdims=["Lng", "Lat"], vdims=["City", "Humidity", "Hotel Name", "Country"])

tiles = gv.tile_sources.OSM()
 
points = gdf.to(gv.Points, ["Lng", "Lat"], ["Humidity", "City", "Hotel Name", "Country"])

output = tiles * points.opts(
    width=1200,
    height=900,
    size="Humidity",
    color="City",
    cmap="viridis",
    tools=['hover']
)

output

c:\Users\anoje\anaconda3\Lib\site-packages\holoviews\core\util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.5, use 'transform_reference' instead.
  value = param_value_if_widget(value)


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)